In [1]:
#Read in Libraries necessary for cleaning

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
import nltk
import seaborn as sns
from spacy.lang.en import English
from langdetect import detect
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
Titles = pd.read_csv('Total_Title_Data.csv')

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_1900\3995532456.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Titles = pd.read_csv('Total_Title_Data.csv')


In [4]:
Titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426870 entries, 0 to 426869
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   EAN                   426870 non-null  int64  
 1   Title                 426870 non-null  object 
 2   Alt Vend              426870 non-null  object 
 3   Ingram Category Code  425121 non-null  object 
 4   MJR BISAC             426870 non-null  object 
 5   BISAC                 426870 non-null  object 
 6   List                  426870 non-null  float64
 7   Annotation            426870 non-null  object 
 8   Category              426870 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 29.3+ MB


In [5]:
Titles['EAN'] = Titles['EAN'].astype('str')
Titles['Title'] = Titles['Title'].astype('str')
Titles['Alt Vend'] = Titles['Alt Vend'].astype('str')
Titles['Ingram Category Code'] = Titles['Ingram Category Code'].astype('str')
Titles['MJR BISAC'] = Titles['MJR BISAC'].astype('str')
Titles['Annotation'] = Titles['Annotation'].astype('str')
Titles['Category'] = Titles['Category'].astype('str')

In [6]:
Titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426870 entries, 0 to 426869
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   EAN                   426870 non-null  object 
 1   Title                 426870 non-null  object 
 2   Alt Vend              426870 non-null  object 
 3   Ingram Category Code  426870 non-null  object 
 4   MJR BISAC             426870 non-null  object 
 5   BISAC                 426870 non-null  object 
 6   List                  426870 non-null  float64
 7   Annotation            426870 non-null  object 
 8   Category              426870 non-null  object 
dtypes: float64(1), object(8)
memory usage: 29.3+ MB


In [7]:
Titles.head()

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,List,Annotation,Category
0,9781803143057,Murder at the Country Club: An absolutely unpu...,6072814,FR,FIC,FIC027110,9.99,<p><strong>Kitty Underhay is playing doubles.....,1
1,9798440995017,From Sandpaper To Silk (Book One) ...,6107230,OP,OCC,OCC000000,15.00,"This is a very suspenseful, engaging, and rive...",1
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,6110949,BA,BIO,BIO000000,9.99,<p>Real Life Story About the Gilgo Beach Murde...,1
3,9781636160887,Izzie's Incredible Imagination ...,9269037,CL,JUV,JUV001000,9.99,<p>Whether you are floating in space or riding...,1
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",6027623,BA,BIO,BIO026000,9.50,"<p>Born in 1909 in Hull, Bernard recalls how h...",1


In [8]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

Titles['Annotation Text'] = Titles['Annotation'].apply(remove_html_tags)

C:\Users\Pupnsuds90\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
Titles.head()

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,List,Annotation,Category,Annotation Text
0,9781803143057,Murder at the Country Club: An absolutely unpu...,6072814,FR,FIC,FIC027110,9.99,<p><strong>Kitty Underhay is playing doubles.....,1,Kitty Underhay is playing doubles... with deat...
1,9798440995017,From Sandpaper To Silk (Book One) ...,6107230,OP,OCC,OCC000000,15.00,"This is a very suspenseful, engaging, and rive...",1,"This is a very suspenseful, engaging, and rive..."
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,6110949,BA,BIO,BIO000000,9.99,<p>Real Life Story About the Gilgo Beach Murde...,1,Real Life Story About the Gilgo Beach Murders ...
3,9781636160887,Izzie's Incredible Imagination ...,9269037,CL,JUV,JUV001000,9.99,<p>Whether you are floating in space or riding...,1,Whether you are floating in space or riding a ...
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",6027623,BA,BIO,BIO026000,9.50,"<p>Born in 1909 in Hull, Bernard recalls how h...",1,"Born in 1909 in Hull, Bernard recalls how he s..."


In [10]:
Titles['Annotation Text'] = Titles['Annotation Text'].astype('str')

In [18]:
#Detecting Language by Title to Eliminate non-English titles from dataset
tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")

papluca = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")

Titles['PredictedLanguage'] = ''
Titles['LanguageConfidence'] = 0.0

def predict_language(text):
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    outputs = papluca(**inputs)
    predicted_language = paplu.decode(outputs.logits.argmax(dim=1))
    confidence = outputs.logits.softmax(dim=1).max().item()
    return predicted_language, confidence

for index, row in Titles.iterrows():
    title = row['Annotation Text']
    
    predicted_language, confidence = predict_language(title)
    
    Titles.at[index, 'PredictedLanguage'] = predicted_language
    Titles.at[index, 'LanguageConfidence'] = confidence
    
        

KeyboardInterrupt: 

In [57]:
pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

In [66]:
pipe()

[{'label': 'es', 'score': 0.9775945544242859}]

In [47]:
predicted_language

'.'

In [45]:
Titles.loc[470,'Annotation Text']

'Victor Scarinzi, a self-described Italian redneck, shares stories that range from daydreaming as a boy, to getting into trouble as a teenager, to learning the lessons of adulthood in this collection of memoirs. It was by spending time in nature that he became convinced that there must be a God, because who else could create the wonderful woods, lakes, mountains, swamps, deserts, and animals that you see in the outdoors? Fishing, hunting, thinking of faraway places, sorting out his dreams, and planning his futureusually with a dog tagging along by his sideare some of what he treasures most. The stories will no doubt make you think of your own happy memories in nature and inspire you to protect the outdoors and all that is in it. Join a simple man as he shares simple dreamsmany of which hes accomplishedand others that he hasnt given up on yet.'

In [20]:
Titles['PredictedLanguage'].value_counts()

        426630
e          216
-           12
</s>         8
n            1
(            1
de           1
a            1
Name: PredictedLanguage, dtype: int64

In [ ]:
import re

#Identify Titles in the datset with the code pattern and remove them from the dataframe
pattern = r'&#\d+;'
Titles = Titles[~Titles['Column'].str.contains(pattern, regex=True)]

#Reset the index of the dataframe
Titles = Titles.reset_index(drop=True)